In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import pathlib

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

print(tf.__version__)

2.0.0


In [2]:
import numpy as np
import matplotlib.pyplot as plt  # To visualize
import pandas as pd  # To read data
from sklearn.linear_model import LinearRegression
data = pd.read_csv('data.csv')
train = pd.read_csv('train.csv')
val = pd.read_csv('val.csv')
test = pd.read_csv('test.csv')



In [3]:
from statistics import mode 
data.fillna(data.mean(), inplace=True)
data['host_since'].fillna(mode(data['host_since']), inplace=True)
data['host_response_time'].fillna(mode(data['host_response_time']), inplace=True)
data['host_response_rate'].fillna(mode(data['host_response_rate']), inplace=True)
data['host_is_superhost'].fillna(mode(data['host_is_superhost']), inplace=True)
data['host_listings_count'].fillna(mode(data['host_listings_count']), inplace=True)
data['host_identity_verified'].fillna(mode(data['host_identity_verified']), inplace=True)
print(data.isnull().sum())

id                                0
host_since                        0
host_response_time                0
host_response_rate                0
host_is_superhost                 0
host_listings_count               0
host_identity_verified            0
zipcode                          94
latitude                          0
longitude                         0
is_location_exact                 0
property_type                     0
room_type                         0
accommodates                      0
bathrooms                         0
bedrooms                          0
beds                              0
bed_type                          0
amenities                         0
square_feet                       0
number_of_reviews                 0
number_of_reviews_ltm             0
first_review                   2007
last_review                    2007
review_scores_rating              0
review_scores_accuracy            0
review_scores_cleanliness         0
review_scores_checkin       

In [4]:

def convert_to_year(date_in_some_format):
    date_as_string = str(date_in_some_format)
    year_as_string = date_as_string[:4] # last four characters
    return int(year_as_string)

def percent_to_float(s):
    s = str(float(s.rstrip("%")))
    i = s.find(".")
    if i == -1:
        return int(s) / 100
    if s.startswith("-"):
        return -percent_to_float(s.lstrip("-"))
    s = s.replace(".", "")
    i -= 2
    if i < 0:
        return float("." + "0" * abs(i) + s)
    else:
        return float(s[:i] + "." + s[i:])
def boolean_to_int(b):
    if b=='t':
        return 0
    elif b=='f':
        return 1
    else:
        return


In [5]:
data['host_since'] = data['host_since'] .apply(convert_to_year)
data['host_response_rate'] = data['host_response_rate'] .apply(percent_to_float)
data['host_is_superhost'] = data['host_is_superhost'] .apply(boolean_to_int)
data['host_identity_verified'] = data['host_identity_verified'] .apply(boolean_to_int)
data['is_location_exact'] = data['is_location_exact'] .apply(boolean_to_int)

print(data)

          id  host_since  host_response_time  host_response_rate  \
0          0        2008  within a few hours                 1.0   
1          1        2008  within a few hours                 1.0   
2          2        2009      within an hour                 1.0   
3          3        2009      within an hour                 1.0   
4          4        2009      within an hour                 1.0   
...      ...         ...                 ...                 ...   
10147  11334        2017      within an hour                 1.0   
10148  11335        2015      within an hour                 1.0   
10149  11336        2019      within an hour                 1.0   
10150  11337        2015      within an hour                 1.0   
10151  11338        2019      within an hour                 1.0   

       host_is_superhost  host_listings_count  host_identity_verified  \
0                      0                  3.0                       0   
1                      0             

In [6]:
pd.set_option('display.max_columns', None)
data.tail()


,id,host_since,host_response_time,host_response_rate,host_is_superhost,host_listings_count,host_identity_verified,zipcode,latitude,longitude,is_location_exact,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,square_feet,number_of_reviews,number_of_reviews_ltm,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,price
10147,11334,2017,within an hour,1.0,1,2.0,1,78746.0,30.25875,-97.78971,0,Apartment,Entire home/apt,8,2.0,2.0,4.0,Real Bed,"{TV,Wifi,""Air conditioning"",Pool,Kitchen,""Free...",1041.262857,0,0,NaN,NaN,96.287935,9.786088,9.625358,9.871562,9.876556,9.783313,9.62637,400.0
10148,11335,2015,within an hour,1.0,1,1.0,1,78745.0,30.19054,-97.77535,1,Apartment,Entire home/apt,2,1.0,1.0,1.0,Real Bed,"{""Air conditioning"",Pool,Kitchen,""Free parking...",1041.262857,1,1,2019-09-18,2019-09-18,96.287935,9.786088,9.625358,9.871562,9.876556,9.783313,9.62637,55.0
10149,11336,2019,within an hour,1.0,1,1.0,1,78744.0,30.18790,-97.76552,1,Apartment,Private room,2,1.0,1.0,1.0,Real Bed,"{""Air conditioning"",Pool,Kitchen,""Smoking allo...",1041.262857,0,0,NaN,NaN,96.287935,9.786088,9.625358,9.871562,9.876556,9.783313,9.62637,50.0
10150,11337,2015,within an hour,1.0,0,1.0,1,78704.0,30.25580,-97.76815,0,Guesthouse,Entire home/apt,2,1.0,1.0,1.0,Real Bed,"{TV,Wifi,""Air conditioning"",Kitchen,""Free park...",1041.262857,0,0,NaN,NaN,96.287935,9.786088,9.625358,9.871562,9.876556,9.783313,9.62637,455.0
10151,11338,2019,within an hour,1.0,1,1.0,0,78735.0,30.25454,-97.83561,0,House,Private room,2,1.0,1.0,1.0,Real Bed,"{TV,Wifi,""Air conditioning"",Kitchen,""Free park...",1041.262857,0,0,NaN,NaN,96.287935,9.786088,9.625358,9.871562,9.876556,9.783313,9.62637,43.0


In [7]:
data_train=data[data.id.isin(train.id)]
y_train=data_train.price
x_train=data_train.drop(columns=['price','zipcode','first_review','last_review','amenities'])



data_val=data[data.id.isin(val.id)]
y_val=data_val.price
x_val=data_val.drop(columns=['price','zipcode','first_review','last_review','amenities'])

data_test=data[data.id.isin(test.id)]
y_test=data_test.price
x_test=data_test.drop(columns=['price','zipcode','first_review','last_review','amenities'])



In [8]:
print(x_train)

          id  host_since  host_response_time  host_response_rate  \
3          3        2009      within an hour                 1.0   
4          4        2009      within an hour                 1.0   
6          6        2009        within a day                 1.0   
7          7        2009  within a few hours                 1.0   
11        11        2010      within an hour                 1.0   
...      ...         ...                 ...                 ...   
10146  11333        2015      within an hour                 1.0   
10147  11334        2017      within an hour                 1.0   
10148  11335        2015      within an hour                 1.0   
10149  11336        2019      within an hour                 1.0   
10151  11338        2019      within an hour                 1.0   

       host_is_superhost  host_listings_count  host_identity_verified  \
3                      0                  1.0                       0   
4                      0             

In [9]:
data=pd.get_dummies(data, columns=["host_response_time"])
data=pd.get_dummies(data, columns=["property_type"])
data=pd.get_dummies(data, columns=["room_type"])
data=pd.get_dummies(data, columns=["bed_type"])
print('done')

done


In [10]:
print(data.id)

0            0
1            1
2            2
3            3
4            4
         ...  
10147    11334
10148    11335
10149    11336
10150    11337
10151    11338
Name: id, Length: 10152, dtype: int64


In [11]:
data.to_csv(r'data_p.csv')